# Model A

In [12]:
# Load required packages
include("../src/EUROml.jl")
using Dates, JLD2, .EUROml

In [13]:
# Load data
results, scorers, shootouts = EUROml.load_data(dir="..")
data, team2id, id2team = EUROml.convert_data(results, scorers, shootouts;output=false)

# Build model
model_build = EUROml.build_model_A(maximum(data.id_home_team))

EuroModel(
    embedding = Embedding(247 => 6),    # 1_482 parameters
    core = Chain(
        layer_1 = Dropout(0.1),
        layer_2 = Dense(14 => 8, relu),  # 120 parameters
        layer_3 = Dense(8 => 4, relu),  # 36 parameters
        layer_4 = Dense(4 => 2),        # 10 parameters
    ),
)         # Total: 1_648 parameters,
          #        plus 2 states.

In [14]:
# Train model
model, stats, tstate = EUROml.train_model(model_build,data;epochs=500,batch_size=1024,name="./model_A")

┌ Warning: No functional GPU backend found! Defaulting to CPU.
│ 
│ 1. If no GPU is available, nothing needs to be done.
│ 2. If GPU is available, load the corresponding trigger package.
│     a. LuxCUDA.jl for NVIDIA CUDA Support.
│     b. LuxAMDGPU.jl for AMD GPU ROCM Support.
│     c. Metal.jl for Apple Metal GPU Support.
└ @ LuxDeviceUtils /home/sebastian/.julia/packages/LuxDeviceUtils/nrcym/src/LuxDeviceUtils.jl:220


InterruptException: InterruptException: